In [99]:
#  Generamos un DataFrame con las columnas Vup Iup Vdow Idow
# la informacion de la muestra esta en el nombre del archivo generado
# ejemplo     nombre-muestra_ciclo_repeticion   
#              HF928_C1_R1

# Update Oct 16
# Nombre archivos: celda1_ciclo_0000.txt     

import pandas as pd
import numpy as np
import sys

# Leer el archivo de texto con pandas
def leer_txt_pandas(filepath):
    # Leer el archivo como texto y eliminar las primeras líneas de encabezado
    with open(filepath, 'r') as file:
        lines = file.readlines()

    #ciclo = lines[0].strip()
    #rep = lines[1].strip()
    # Extraer la fecha y hora
    date = lines[3].strip()  # Fecha en la línea 4
    time = lines[4].strip()  # Hora en la línea 5
    
    # Crear columna datetime
    datetime = pd.to_datetime(f"{date} {time}", format="%d-%m-%Y %H:%M:%S")
    
    # Leer datos de DIR UP (líneas 7 a 56) y DIR DOWN (líneas 63 a 113)
    df_up = pd.read_csv(filepath, sep=r'\s+', skiprows=6, nrows=50, names=["Vup", "Iup"])
    df_down = pd.read_csv(filepath, sep=r'\s+', skiprows=63, nrows=50, names=["Vdw", "Idw"])
    
    # Agregar la columna datetime
    df_up['datetime'] = datetime
   # df_down['datetime'] = datetime

    # Combinar ambos DataFrames
    df_final = pd.concat([df_up.reset_index(drop=True), df_down.reset_index(drop=True)], axis=1)
    
    return (df_final)     #, ciclo, rep])

In [100]:
# Loop para crear 80 dfs correspondientes a los primeros 80 ciclos

num_celda = ['1','2']

lista_df_celda=[]
for j in num_celda:                         #loop sobre las celdas
    lista_df=[]
    for i in range(80):                     #loop sobre los ciclos
        ciclo = f'{i:04d}'                  #formato según el nombre de los archivos de SplitData
      
        filepath = 'SplitData/celda'+j+'_ciclo_'+ ciclo + '.txt'            

        # Llamar la función para procesar el archivo
        df1 = leer_txt_pandas(filepath)                            #df1 , ciclo , rep = leer_txt_pandas(filepath)   

        # Poner la columna datetime como index      
        df1.set_index('datetime', inplace=True)     

        num_rows = len(df1)         
        increments = pd.to_timedelta(4 * np.arange(0, len(df1) * 0.01, 0.01), unit='s')         # Revisar incremento
        df1.index = df1.index + increments  # Sumar los incrementos al índice     

        # Convertir las las ',' por '.' 
        df1[['Vup', 'Iup', 'Vdw', 'Idw']] = df1[['Vup', 'Iup', 'Vdw', 'Idw']].replace(',', '.', regex=True)

        # Convertir las columnas 'Vup', 'Iup', 'Vdw' e 'Idw' a tipo float
        df1[['Vup', 'Iup', 'Vdw', 'Idw']] = df1[['Vup', 'Iup', 'Vdw', 'Idw']].astype(float)

        lista_df.append(df1)
    lista_df_celda.append(lista_df)

In [101]:
def zeroMask(df1, var):
    return (df1[var] - 0.).abs().idxmin()                            #index of the 'var' value closest to zero 

def tangetMask(df1, x_var, y_var):
    x = df1[x_var] ; y = df1[y_var]
    return (np.abs(np.gradient(x, y/Ama) - 1.)).argmin()             #index where derivative of y_var w/r/ to x_var is closest to 1         

def productMask(df2, x_var, y_var):
    df3 = df2[df2[y_var] < 0]                                   # filter negative currents
    df3['product'] = (df3[x_var] * df3[y_var]).abs()            # add a new column to the local df
    return df3['product'].idxmax()

#Define functions: FF & PCE for each run ; HI for each cicle (i.e. up and down runs)

def FF(Vmax, jmax, Voc, jSC):
    return Vmax*jmax / Voc*jSC                      # Adimensional

def PCE(Vmax, jmax, Pin=100.0):                       # Eficiencia
    return np.abs(100.0*Vmax*jmax / Pin)              # [%]  ;  Pin: dato piranómetro ; (lab) P_In = 100 mW/cm^2 = 100 V*mA/cm^2

def HI(jMaxFor, jMaxRev):
    return 1.0 + jMaxFor / jMaxRev 

In [102]:
#Hallar Parámetros fotovoltaicos 

pd.options.mode.copy_on_write = True

Ama_list = [1.0, 20.25]

for j in range(len(num_celda)):                    #loop sobre celdas
    lista_df = lista_df_celda[j]            #lista de df correspondientes a la celda j
    data = []

    for i in range(80):                     #loop sobre ciclos
        df2 = lista_df[i]
        df = df2.rename(columns={"Iup": "iUp", "Idw": "iDw"})               #Adapt to local names   

        df[['iUp', 'iDw']] = df[['iUp', 'iDw']]*1000                    # [A] to [mA]    
            
        Ama = Ama_list[j]

        jSC_up = df['iUp'][zeroMask(df, 'Vup')]  / Ama                  # [mA/cm²]  
        jSC_dw = df['iDw'][zeroMask(df, 'Vdw')]  / Ama

        Voc_up = df['Vup'][zeroMask(df, 'iUp')]                         # [V]
        Voc_dw = df['Vdw'][zeroMask(df, 'iDw')] 

        Vmax_up = df['Vup'][productMask(df, 'Vup', 'iUp')]                
        jMax_up = df['iUp'][productMask(df, 'Vup', 'iUp')] / Ama          
            
        Vmax_dw = df['Vdw'][productMask(df, 'Vdw', 'iDw')]                
        jMax_dw = df['iDw'][productMask(df, 'Vdw', 'iDw')] / Ama          

        PCE_dw = round(PCE(Vmax_dw, jMax_dw),4) ; PCE_up = round(PCE(Vmax_up, jMax_up),4)
        FF_dw = round(FF(Vmax_dw, jMax_dw, Voc_dw, jSC_dw), 4) ; FF_up = round(FF(Vmax_up, jMax_up, Voc_up, jSC_up), 4)

        hi = round(HI(jMax_up, jMax_dw),4)

        lista_parametros = [i+1, Voc_up, jSC_up, jMax_up ,Vmax_up, PCE_up, FF_up,
                            Voc_dw, jSC_dw, jMax_dw, Vmax_dw, PCE_dw, FF_dw, hi]

        data.append(lista_parametros)    

    df = pd.DataFrame(data,columns=['ciclo','Voc up', 'jSC up', 'jMax up', 'Vmax up', 'PCE up', 'FF up',
                                    'Voc dw', 'jSC dw', 'jmax dw','Vmax dw', 'PCE dw', 'FF dw', 'HI'])
    df.to_pickle('celda'+str(j+1)+'_80ciclos.pkl')



In [103]:
# Exportar df combinado

df1 = pd.read_pickle('celda1_80ciclos.pkl').set_index('ciclo')
df2 = pd.read_pickle('celda2_80ciclos.pkl').set_index('ciclo')

# Create a MultiIndex
arrays = [
    df1.index.tolist() * 2, ['1'] * len(df1) + ['2'] * len(df2)]

# Intercalate the dfs
df_combined = pd.concat([df1, df2]).sort_index(kind='stable')

df_combined.index = pd.MultiIndex.from_tuples(
    [(idx, i) for idx in df_combined.index.unique() for i in range(1, (df_combined.index == idx).sum() + 1)],
    names=['Ciclo', 'Celda'])

df_combined#.to_pickle("80ciclos.pkl")

Voc up    jSC up   jMax up  Vmax up  PCE up   FF up  Voc dw  \
Ciclo Celda                                                                
1     1      1.1515 -1.598000 -1.184000   0.6535  0.7737  1.0738  1.2065   
      2      2.4235 -2.298420 -1.727160   1.5940  2.7531  2.6110  2.4235   
2     1      1.1515 -1.685000 -1.242000   0.6535  0.8116  1.1877  1.2065   
      2      2.4235 -2.532395 -1.842420   1.5940  2.9368  3.0688  2.4235   
3     1      1.0960 -1.564000 -1.169000   0.6535  0.7639  1.0902  1.1515   
...             ...       ...       ...      ...     ...     ...     ...   
78    2      2.3680 -1.810914 -1.400049   1.5385  2.1540  1.6472  2.3680   
79    1      1.0410 -0.893000 -0.668000   0.5985  0.3998  0.3430  1.0960   
      2      2.3680 -1.405926 -1.086716   1.5940  1.7322  1.0285  2.3680   
80    1      1.0410 -0.622000 -0.468000   0.5985  0.2801  0.1674  1.0410   
      2      2.3680 -0.974173 -0.752494   1.6490  1.2409  0.5105  2.3680   

               jSC dw   jmax dw  Vmax dw  PCE dw   FF dw      HI  
Ciclo Celda                                                       
1     1     -1.260000 -0.811000   0.6535  0.5300  0.5535  2.4599  
      2     -2.299309 -1.738272   1.5940  2.7708  2.6288  1.9936  
2     1     -1.353000 -0.891000   0.6535  0.5823  0.6530  2.3939  
      2     -2.485877 -1.829778   1.5940  2.9167  2.9917  2.0069  
3     1     -1.324000 -0.791000   0.7090  0.5608  0.6448  2.4779  
...               ...       ...      ...     ...     ...     ...  
78    2     -1.813136 -1.353185   1.5940  2.1570  1.6516  2.0346  
79    1     -0.731000 -0.430000   0.5985  0.2574  0.1716  2.5535  
      2     -1.407358 -1.049531   1.6490  1.7307  1.0286  2.0354  
80    1     -0.518000 -0.298000   0.5985  0.1784  0.0887  2.5705  
      2     -0.974173 -0.750222   1.6490  1.2371  0.5089  2.0030  

[160 rows x 13 columns]